In [2]:
import numpy as np
import os
import json
import random
import shutil
import pathlib
import create_directory

# Khảo sát dữ liệu

In [2]:
feature_path = os.path.join(create_directory.vn_data_dir, "feature")
num_feature_label_dict = {}
for i in os.listdir(feature_path):
    if i!=".DS_Store":
        num_feature_label_dict[i] = len(os.listdir(os.path.join(feature_path, i)))

In [3]:
count=0
for label, num in num_feature_label_dict.items():
    if num>8:
        count+=1
count

710

Những nhãn nào có số feature vector > 8 ta sẽ giữ nhãn lại, còn lại ta sẽ chuyển thành unknown

In [3]:
def copy_file(source_path, target_path, label_count):
    for i in os.listdir(source_path):
        if i!=".DS_Store":
            source_label_path = os.path.join(source_path, str(i))
            if label_count[i]>8:
                target_label_path = os.path.join(target_path, str(i))
                if not os.path.exists(target_label_path):
                    os.makedirs(target_label_path)
                for j in os.listdir(source_label_path):
                    if j!=".DS_Store":
                        shutil.copyfile(os.path.join(source_label_path, j), os.path.join(target_label_path, j))
            else:
                target_label_path = os.path.join(target_path, "Unknown")
                if not os.path.exists(target_label_path):
                    os.makedirs(target_label_path)
                for j in os.listdir(source_label_path):
                    if j!=".DS_Store":
                        j_preprocess = pathlib.Path(j).stem+"_"+str(i)+pathlib.Path(j).suffix
                        shutil.copyfile(os.path.join(source_label_path, j), os.path.join(target_label_path, j_preprocess))

In [5]:
sample_feature_path = os.path.join(create_directory.vn_data_dir, "sample_feature")
if not os.path.exists(sample_feature_path):
    os.makedirs(sample_feature_path)
copy_file(feature_path, sample_feature_path, num_feature_label_dict)

# Tạo dữ liệu feature sample

In [25]:
# Face Database
sample_feature_path = os.path.join(create_directory.vn_data_dir, "sample_feature")
count = 0
train_label_idx = []
train_feature = []
test_label_idx = []
test_feature = []
label2idx = {}
for i in os.listdir(sample_feature_path):
    if i == ".DS_Store":
        continue
    # Ta bỏ qua tất cả các data có nhãn Unknown
    if i == "Unknown":
        continue
    label2idx[i]=count
    list_path = os.listdir(os.path.join(sample_feature_path, i))
    # Trộn thứ tự các feature vector
    random.shuffle(list_path)
    count2 = 0
    for j in list_path:
        if j == ".DS_Store":
            continue
        # Ta chỉ lấy 8 ảnh đầu ngẫu nhiên làm tập cơ sở, còn những ảnh còn lại cho vào test
        feat=np.load(os.path.join(os.path.join(sample_feature_path, i), j))
        if count2<8:
            train_feature.append(feat.reshape(128).tolist()) 
            train_label_idx.append(count)
            count2+=1
        else:
            test_feature.append(feat.reshape(128).tolist()) 
            test_label_idx.append(count)
    count+=1

In [26]:
label2idx["Unknown"]=len(label2idx)
with open(os.path.join(create_directory.vn_data_dir, "label2idx.json"), "w") as outfile:
    json.dump(label2idx, outfile)

In [27]:
# Ta lấy 60 ảnh unknown bất kì
unknown_list_path = os.listdir(os.path.join(sample_feature_path, "Unknown"))
random.shuffle(unknown_list_path)
for idx, i in enumerate(unknown_list_path):
    if i!=".DS_Store":
        if idx>=60:
            break
        feat=np.load(os.path.join(os.path.join(sample_feature_path, "Unknown"), i))
        test_feature.append(feat.reshape(128).tolist()) 
        test_label_idx.append(label2idx["Unknown"])

In [28]:
fixed_threshold_path = os.path.join(create_directory.vn_data_dir, "fixed_threshold")
if not os.path.exists(fixed_threshold_path):
    os.makedirs(sample_feature_path)
with open(os.path.join(fixed_threshold_path, 'train_feature.json'), 'w') as f:
    json.dump(train_feature, f)
with open(os.path.join(fixed_threshold_path, 'test_feature.json'), 'w') as f:
    json.dump(test_feature, f)
with open(os.path.join(fixed_threshold_path, 'train_label_idx.json'), 'w') as f:
    json.dump(train_label_idx, f)
with open(os.path.join(fixed_threshold_path, 'test_label_idx.json'), 'w') as f:
    json.dump(test_label_idx, f)